In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/mistral-7b-32768-fpf')

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/mistral-7b-32768-fpf',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt = '<s>Soalan: KWSP tu apa?\nJawapan:'
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[    1,  1537,   282,   276, 28747,   524, 28780,  3701,  8582,   979,
         28708, 28804,    13, 28798,  1067,  4209, 28747]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [5]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Soalan: KWSP tu apa?
Jawapan: kwsp ialah kwsp iaitu kumpulan wang simpanan pekerja</s>


In [6]:
prompt = '<s>Soalan: kenapa malaysia suka makan nasi lemak?\nJawapan:'
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[    1,  1537,   282,   276, 28747,   446,   269, 10405,  6125,   748,
           515,   519,  2117,   290,   491,   276,   307,  8608,   462, 28719,
           491, 28804,    13, 28798,  1067,  4209, 28747]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}

In [7]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Soalan: kenapa malaysia suka makan nasi lemak?
Jawapan:Nasi Lemak merupakan antara sarapan pagi yang paling popular dalam kalangan rakyat Malaysia terutamanya orang Melayu. Ia mudah didapati di mana-mana sahaja seperti restoran mamak, gerai tepi jalan, gerai di pasar pagi, di stesen minyak, pasar malam, dan sebagainya. Sajian ini juga dikenali sebagai Nasi lemak antarabangsa kerana mendapat sambutan hangat daripada seluruh dunia. Selain itu, nasi lemak mempunyai bau yang sangat menyelerakan dan rasanya juga sangat unik. Tidak hairanlah mengapa ia digemari oleh ramai orang.Nasi lemak mempunyai santan, kerisik, dan serbuk kunyit yang ditumis dalam lemak kelapa dan dibungkus dalam daun pisang atau mangkuk plastik. Nasi lemak biasanya disajikan bersama ayam goreng, daging rendang, telur rebus, ikan bilis, timun, dan sambal yang dibuat daripada kacang tanah, udang atau petai. Variasi lain termasuk parutan kelapa muda, bilis goreng, udang goreng, dan sotong goreng. Di Malaysia, nasi lema

In [8]:
prompt = '<s>Soalan: kerajaan Madani bagus x?\nJawapan:'
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[    1,  1537,   282,   276, 28747, 22399, 12209,   276,  5311,  4499,
          6635,   381,  1318, 28804,    13, 28798,  1067,  4209, 28747]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [9]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
<s> Soalan: kerajaan Madani bagus x?
Jawapan: soalan ni bagus. nak cakap bagus ke ape ke boleh cite sikit. Apa yang diaorang buat? Tapi dalam soalan ni la, ramai yang tak boleh jawab, lagi-lagi untuk soalan tu sahaja. Sebab soalan tu terlampau open ended dengan tiada langsung rujukan yang berkaitan. Hampir 99% gagal menjawab soalan ni. Dan saya memang respect kepada mereka yang bertungkus lumus untuk belajar. Tapi tahun ni sangat berbeza kerana mereka bertungkus-lumus untuk menghafal formula dan rumus. Kalau kita nak hafal semua, memang tidak mampu langsung untuk jawab soalan macam ni. Ramai yang patah balik apabila dapat soalan terbuka macam ni. Ada pula yang terbaca ayat dalam soalan, kemudian sebab dia terangkan terlalu detail tentang tu, dia terlepas pandang dan terabai soalan seterusnya. Huhu. Jangan main-main tau soalan ni, kalau jawab pun berdas

In [12]:
prompt = '<s>Soalan: macam mana nak install awscli dalam ubuntu\nJawapan:'
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[    1,  1537,   282,   276, 28747,  8338,   314,   676, 28708,   307,
           491,  4430, 20933, 18008,  7517,   314, 16179,  2794, 28718,    13,
         28798,  1067,  4209, 28747]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [15]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Soalan: macam mana nak install awscli dalam ubuntu
Jawapan: Anda boleh memasang awscli dalam Ubuntu dengan menggunakan pengurus pakej apt-get. Caranya adalah seperti berikut: 1. Buka terminal dan kemas kini senarai pakej dengan menjalankan arahan berikut: ```sudo apt-get update``` 2. Pasang awscli dengan menjalankan arahan berikut: ```sudo apt-get install awscli``` Selepas menyelesaikan langkah-langkah di atas, anda akan mempunyai awscli dipasang dalam Ubuntu anda. Anda boleh menggunakannya untuk berinteraksi dengan AWS SDK dalam kod Python anda.</s>
